In [ ]:
from IPython.core.display import HTML
with open ('style.css', 'r') as file:
    css = file.read()
HTML(css)

# How to Check that a Formula is a Tautology

In this notebook we develop a function <tt>tautology</tt> that takes a formula $f$ from propositional logic and checks whether $f$ is a tautology.  As we represent tautologies as nested tuples, we first have to import the parser for propositional logic.

In [ ]:
import propLogParser as plp

As we represent propositional valuations as sets of variables, we need a function to compute all subsets of a given set.  The module <tt>power</tt> provides a function called <tt>allSubsets</tt> such that for a given set $M$ the function call $\texttt{allSubsets}(M)$ computes a list containing all subsets of $M$, that is we have:
$$ \texttt{allSubsets}(M) = \bigl[A \mid A \in 2^M\bigr] $$

In [ ]:
import power

In [ ]:
power.allSubsets({'p', 'q'})

To be able to compute all propositional valuations for a given formula $f$ we first need to determine the set of all variables that occur in $f$.  The function $\texttt{collectVars}(f)$ takes a formula $f$ from propositional logic and computes all propositional variables occurring in $f$.  This function is defined recursively.

In [ ]:
def collectVars(f):
    "Collect all propositional variables occurring in the formula f."
    if f[0] in ['⊤', '⊥']:
        return set()
    if isinstance(f, str):
        return { f }
    if f[0] == '¬':
        return collectVars(f[1])
    return collectVars(f[1]) | collectVars(f[2]) 

We have discussed the function <tt>evaluate</tt> previously.  The call 
$\texttt{evaluate}(f, I)$ takes a propsitional formula $f$ and a propositional valuation $I$, where $I$ is represented as a set of propositional variables.  It evaluates $f$ given $I$.

In [ ]:
def evaluate(f, I):
    """
    Evaluate the propositional formula f using the propositional valuation I.
    I is represented as a set of variables.
    """
    if isinstance(f, str):
        return f in I
    if f[0] == '⊤': return True
    if f[0] == '⊥': return False
    if f[0] == '¬': return not evaluate(f[1], I)
    if f[0] == '∧': return evaluate(f[1], I) and evaluate(f[2], I)
    if f[0] == '∨': return evaluate(f[1], I) or  evaluate(f[2], I)
    if f[0] == '→': return not evaluate(f[1], I) or evaluate(f[2], I)
    if f[0] == '↔': return evaluate(f[1], I) == evaluate(f[2], I)

Now we are ready to define the function $\texttt{tautology}(f)$ that takes a propositional formula $f$ and checks whether $f$ is a tautology.  If $f$ is a tautology, the function returns <tt>True</tt>, otherwise a set of variables $I$ is returned such that $f$ evaluates to <tt>False</tt> if all variables in $I$ are <tt>True</tt>, while all variables not in $I$ are <tt>False</tt>.

In [ ]:
def tautology(f):
    "Check, whether the formula f is a tautology."
    P = collectVars(f)
    A = power.allSubsets(P)
    if all(evaluate(f, I) for I in A):
        return True
    else:
        return [I for I in A if not evaluate(f, I)][0]   

The function $\texttt{test}(s)$ takes a string $s$ that can be parsed as a propositionl formula and checks whether this formula is a tautology.

In [ ]:
def test(s):
    f = plp.LogicParser(s).parse()
    counterExample = tautology(f);
    if counterExample == True: 
        print('The formula', s, 'is a tautology.')
    else: 
        P = collectVars(f)
        print('The formula ', s, ' is not a tautology.')
        print('Counter example: ')
        for x in P:
            if x in counterExample:
                print(x, "↦ True")
            else:
                print(x, "↦ False")

Let us run a few tests.

The first example is DeMorgan's rule.

In [ ]:
test('¬(p ∨ q) ↔ ¬p ∧ ¬q')

In [ ]:
test('(p → q) → (¬p → q) → q')

In [ ]:
test('(p → q) → (¬p → ¬q)')

In [ ]:
test('¬p ↔ (p → ⊥)')